In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
from torch.optim import SGD,Adam,lr_scheduler

import torchvision
import torchvision.transforms as transforms

import os
#import argparse

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
print(device)

cuda


In [ ]:
print('==> Preparing data..')
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30), # Augmentation step
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

==> Preparing data..


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


In [ ]:
class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        dropout=0.25
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.dropout = nn.Dropout(dropout)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out = self.dropout(out)
        out += self.shortcut(x)
        out = F.relu(out)
        return out


In [ ]:
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
net = ResNet18()

In [ ]:
pytorch_total_params = sum(p.numel() for p in net.parameters())
print("Total_params",pytorch_total_params)

Total_params 11173962


In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(),lr=0.001,
                      momentum=0.9, weight_decay=5e-4)

scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)

In [ ]:
net = net.to(device)
if device == 'cuda':
    net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

In [ ]:
logFIleName = 'log_aug_norm_SGD_dropout.txt'

In [ ]:
def train(epoch):
    print('\nEpoch: %d' % epoch, file=open(logFIleName,'a'))
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    running_loss = 0
    running_acc = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        # print statistics
        running_loss += loss.item() * inputs.size(0)
        out = torch.argmax(outputs.detach(),dim=1)
        assert out.shape==targets.shape
        running_acc += (targets==out).sum().item()
    print("Training loss:", running_loss/total,file=open(logFIleName,'a'))
    print("Training loss:", running_loss/total)
    print("Training  accuracy is :", 100.*running_acc/total,file=open(logFIleName,'a'))
    print("Training  accuracy is :", 100.*running_acc/total)

In [ ]:
def test(epoch):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    acc = 100.*correct/total
    print('The test accuracy is :', acc,file=open(logFIleName,'a'))
    print('The test accuracy is :', acc)
    print('best accuracy :', best_acc)
    if acc > best_acc:
      print('Saving..')
      state = {
          'net': net.state_dict(),
          'acc': acc,
          'epoch': epoch,
        }
      if not os.path.isdir('checkpoint'):
          os.mkdir('checkpoint')
      torch.save(net.module.state_dict(),'./checkpoint/saved_module.pt')
      best_acc = acc

In [ ]:
for epoch in range(0,200):
    train(epoch)
    test(epoch)


Epoch: 0
Training loss: 1.807228120651245
Training  accuracy is : 32.548
The test accuracy is : 41.65
best accuracy : 0
Saving..

Epoch: 1
Training loss: 1.5017151314544677
Training  accuracy is : 44.648
The test accuracy is : 45.26
best accuracy : 41.65
Saving..

Epoch: 2
Training loss: 1.3456212731552124
Training  accuracy is : 50.978
The test accuracy is : 52.57
best accuracy : 45.26
Saving..

Epoch: 3
Training loss: 1.230546114883423
Training  accuracy is : 55.406
The test accuracy is : 55.56
best accuracy : 52.57
Saving..

Epoch: 4
Training loss: 1.1425091380500794
Training  accuracy is : 59.056
The test accuracy is : 62.75
best accuracy : 55.56
Saving..

Epoch: 5
Training loss: 1.076455372657776
Training  accuracy is : 61.45
The test accuracy is : 65.45
best accuracy : 62.75
Saving..

Epoch: 6
Training loss: 1.0283614814376831
Training  accuracy is : 63.308
The test accuracy is : 67.57
best accuracy : 65.45
Saving..

Epoch: 7
Training loss: 0.981757853717804
Training  accuracy i